# Predictive Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, tree, neighbors, linear_model, ensemble
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc, mean_squared_error, accuracy_score
import math

In [2]:
data = pd.read_csv("bechdel_analysis_data_cleaned.csv", float_precision=None)
data

,year,imdb,title,clean_test,budget_2013$,binary_PASS,domgross_2013$,intgross_2013$,genres,prod_comps,overview,perc_women_producers,perc_women_directors
0,2013,tt1711425,21 &amp; Over,notalk,13000000,0,25682380.0,42195766.0,['Comedy'],"['Mandeville Films', 'Relativity Media', 'SkyL...",Brilliant student Jeff Chang has the most impo...,0.00,0.0
1,2012,tt1343727,Dredd 3D,ok,45658735,1,13611086.0,41467257.0,"['Action', 'Science Fiction']","['Rena Film', 'DNA Films', 'Reliance Big Enter...","In the future, America is a dystopian wastelan...",0.00,0.0
2,2013,tt2024544,12 Years a Slave,notalk,20000000,0,53107035.0,158607035.0,"['Drama', 'History']","['New Regency Pictures', 'Plan B Entertainment...","In the pre-Civil War United States, Solomon No...",0.25,0.0
3,2013,tt1272878,2 Guns,notalk,61000000,0,75612460.0,132493015.0,"['Action', 'Comedy', 'Crime']","['Universal Pictures', 'Marc Platt Productions...",A DEA agent and an undercover Naval Intelligen...,0.00,0.0
4,2013,tt0453562,42,men,40000000,0,95020213.0,95020213.0,['Drama'],"['Warner Bros. Pictures', 'Legendary Pictures']","The powerful story of Jackie Robinson, the leg...",0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,1971,tt0067741,Shaft,notalk,305063707,0,404702718.0,616827003.0,"['Action', 'Crime', 'Thriller']","['Shaft Productions', 'Metro-Goldwyn-Mayer']",Cool black private eye John Shaft is hired by ...,0.00,0.0
1723,1971,tt0067800,Straw Dogs,notalk,143862856,0,59412143.0,64760273.0,"['Crime', 'Drama', 'Thriller']",['ABC Pictures'],"David Sumner, a mild-mannered academic from th...",0.00,0.0
1724,1971,tt0067116,The French Connection,notalk,12659931,0,236848653.0,236848653.0,"['Action', 'Crime', 'Thriller']","[""D'Antoni Productions"", 'Schine-Moore Product...",Tough narcotics detective 'Popeye' Doyle is in...,0.00,0.0
1725,1971,tt0067992,Willy Wonka &amp; the Chocolate Factory,men,17263543,0,23018057.0,23018057.0,"['Family', 'Fantasy', 'Comedy']",['David L. Wolper Productions'],When eccentric candy man Willy Wonka promises ...,0.00,0.0


In [3]:
genre_data = data.genres
prod_comp_data = data.prod_comps
overview_data = data.overview

data = data.drop(['genres', 'prod_comps', 'overview', 'title', 'imdb'], axis=1)

In [4]:
Y = data.binary_PASS
data = data.drop(['binary_PASS'], axis=1)
X = data

In [16]:
le = preprocessing.LabelEncoder()

le.fit(X.clean_test)
list(le.classes_)
le.transform(X.clean_test)
X.clean_test = 

array([2, 4, 2, ..., 2, 1, 4])

In [5]:
X

,year,clean_test,budget_2013$,domgross_2013$,intgross_2013$,perc_women_producers,perc_women_directors
0,2013,notalk,13000000,25682380.0,42195766.0,0.00,0.0
1,2012,ok,45658735,13611086.0,41467257.0,0.00,0.0
2,2013,notalk,20000000,53107035.0,158607035.0,0.25,0.0
3,2013,notalk,61000000,75612460.0,132493015.0,0.00,0.0
4,2013,men,40000000,95020213.0,95020213.0,0.00,0.0
...,...,...,...,...,...,...,...
1722,1971,notalk,305063707,404702718.0,616827003.0,0.00,0.0
1723,1971,notalk,143862856,59412143.0,64760273.0,0.00,0.0
1724,1971,notalk,12659931,236848653.0,236848653.0,0.00,0.0
1725,1971,men,17263543,23018057.0,23018057.0,0.00,0.0


In [6]:
Y

0       0
1       1
2       0
3       0
4       0
       ..
1722    0
1723    0
1724    0
1725    0
1726    1
Name: binary_PASS, Length: 1727, dtype: int64

In [13]:
X.clean_test.unique()

array(['notalk', 'ok', 'men', 'nowomen', 'dubious'], dtype=object)

In [7]:
# Question 1.a.i - Decision Tree Classifier
dt_accuracy = {1: None, 5: None, 10: None, 50: None, 100: None}
for i in dt_accuracy:
    # Train decision tree classifier with given max_depth
    dt_clf = tree.DecisionTreeClassifier(max_depth=i)
    dt_accuracy[i] = cross_val_score(dt_clf, X, Y, cv=5).mean()

C:\Users\alexa\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\alexa\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\alexa\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\alexa\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 165, in fit
    X, y = self._validate_data(
  File "C:\Users\alexa\anaconda3\lib\site-packages\sklearn\base.py", line 